<a href="https://colab.research.google.com/github/sanja7s/AI_in_EO_Human_Rights/blob/main/EO_Human_Rights_Prompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup
#### Load the API key and relevant Python libaries.

In [ ]:
%%capture
!pip install langchain
!pip install faiss-cpu
!pip install openai==0.28
!pip install unstructured
!pip install python-dotenv
!pip install tiktoken

In [ ]:
from google.colab import files
import io
from dotenv import dotenv_values, load_dotenv, find_dotenv
import openai
import os
from copy import deepcopy
import json
import time
import ast

import pprint

# env file
uploaded = files.upload()


Saving env_HumanRights to env_HumanRights


In [ ]:
# Get the first key from the uploaded dictionary
env_file_key = list(uploaded.keys())[0]

# Read the uploaded file
env_content = uploaded[env_file_key].decode('utf-8')

# Load the content into a variable
env_variables = dotenv_values(stream=io.StringIO(env_content))

api_key = env_variables['OPENAI_API_KEY']
openai.api_key = api_key

# Models

In [ ]:
def get_completion_from_messages(messages,
                                 model="gpt-4",
                                 temperature=0,
                                 max_tokens=1600): #1100
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
        max_tokens=max_tokens, # the maximum number of tokens the model can ouptut
    )
    return response.choices[0].message["content"]

In [ ]:
def get_completion_and_token_count(messages,
                                 model="gpt-4",
                                 temperature=0,
                                 max_tokens=1600):

    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )

    content = response.choices[0].message["content"]

    token_dict = {
    'prompt_tokens':response['usage']['prompt_tokens'],
    'completion_tokens':response['usage']['completion_tokens'],
    'total_tokens':response['usage']['total_tokens'],
    }

    return content, token_dict

# Iterate and Save Use Riskiness Results

# Functions

In [ ]:
def replace_key(d, old_key, new_key):
  """
  Replace `old_key` with `new_key` in dictionary `d`.
  The associated value is retained.
  """
  if old_key in d:
      d[new_key] = d.pop(old_key)
  return d

## Read In Prompt Result

In [ ]:
def read_prompt_output():
  print("Select the right input you need.")
  selected_prompt_uploaded = files.upload() # change this for other prompts

  # Get the first key from the uploaded dictionary
  file_key = list(selected_prompt_uploaded.keys())[0]

  # Read the uploaded file
  file_content = selected_prompt_uploaded[file_key].decode('utf-8')

  file_content_dict = ast.literal_eval(file_content)

  N = len(file_content_dict[0])

  # rename the uses so that we have 46 ids
  i = 0
  for el in file_content_dict:
    for use_el in el:
      use_el['Use'] = int(use_el['Use']) + i * N
    i += 1

  flattened_list = [item for sublist in file_content_dict for item in sublist]
  return flattened_list

In [ ]:
prompt_result = read_prompt_output()

Select the right input you need.


Saving EO_uses_138_FINAL.json to EO_uses_138_FINAL.json


In [ ]:
prompt_result

[{'Use': 1,
  'Domain': 'Biometric identification and categorization of natural persons',
  'Purpose': 'Identifying individuals in crowded areas',
  'Capability': 'Facial recognition from aerial footage',
  'Sensing instrument and location': 'Optical cameras on drones',
  'AI User': 'Law enforcement agencies',
  'AI Subject': 'Individuals in public spaces'},
 {'Use': 2,
  'Domain': 'Biometric identification and categorization of natural persons',
  'Purpose': 'Tracking wildlife for conservation efforts',
  'Capability': 'Animal species identification and tracking',
  'Sensing instrument and location': 'Infrared cameras on satellites',
  'AI User': 'Conservation organizations',
  'AI Subject': 'Wildlife species'},
 {'Use': 3,
  'Domain': 'Biometric identification and categorization of natural persons',
  'Purpose': 'Monitoring migration patterns',
  'Capability': 'Population movement tracking',
  'Sensing instrument and location': 'Radar on satellites',
  'AI User': 'Government agencies

# PART 1 PROMPT: HR 1-10

In [ ]:
# Assuming you have the variables domain, purpose, aiCapability, aiUser, and aiSubject defined with appropriate values

MESSAGES = [
    {
        'role': 'system',
        'content': """You are a renowned specialist in the field of Earth Observation technology with a dedicated focus on understanding, promoting, and supporting the Universal Human Rights. With your vast experience, decisiveness, and conscientious approach, you have a deep understanding of how Earth Observation technologies can be leveraged to impact the Human Rights.
        You possess comprehensive knowledge of the List of human rights definitions, which encapsulates all 30 articles from the UN Universal Decalartion on Human Rights. This global framework was meticulously crafted by the United Nations.
        """
    },
    {
        'role': 'user',
        'content': """

        Assess the AI system's alignment with the Universal Declaration of Human Rights. Specifically, for each of the human rights listed and explained below, assess if the AI system supports it, violates it, or does not impact it.

        Follow the streamlined steps below:
        1. Describe the AI System: Craft a concise description of the AI system, ensuring it parallels the phrasing used in the Sustainable Development Goal targets and indicators. Your description should begin with "The AI system is designed to..." and should be encapsulated within two sentences.
        2. Evaluate Each human right (described in the article). Classify the impact on human rights as "Supports", "Violates", or "Does not impact". To do so, assess if the AI system:
            i) Supports it, i.e., presents an opportunity for or supports the respective human right,
            ii) Violates it, i.e., presents a risk for the respective human right,
            iii) Does not impact it, i.e., there is no clear link that the system supports or violated the respective human right.

        3. In each case, rigorously validate your reasoning. While evaluating, explicitly cite the article text that resonate most with the AI system's purpose and capabilities. Reference the exact text from the respective human right article and provide a thorough rationale. The true intent and capabilities of the AI system are crucial for this evaluation. Ensure there is a direct correlation between the system's functionalities and the referenced article.

        This assessment requires a deep understanding of both the AI system in question and the articles of the Universal Declaration of Human Rights. Each evaluation must be backed by solid reasoning and a clear understanding of the interplay between technology and human rights.
        Remember, precision is paramount. Be succinct.


        *** Universal Declaration of Human Rights ***

        Preamble
        Whereas recognition of the inherent dignity and of the equal and inalienable rights of all members of the human family is the foundation of freedom, justice and peace in the world,
        Whereas disregard and contempt for human rights have resulted in barbarous acts which have outraged the conscience of mankind, and the advent of a world in which human beings shall enjoy freedom of speech and belief and freedom from fear and want has been proclaimed as the highest aspiration of the common people,
        Whereas it is essential, if man is not to be compelled to have recourse, as a last resort, to rebellion against tyranny and oppression, that human rights should be protected by the rule of law,
        Whereas it is essential to promote the development of friendly relations between nations,
        Whereas the peoples of the United Nations have in the Charter reaffirmed their faith in fundamental human rights, in the dignity and worth of the human person and in the equal rights of men and women and have determined to promote social progress and better standards of life in larger freedom,
        Whereas Member States have pledged themselves to achieve, in co-operation with the United Nations, the promotion of universal respect for and observance of human rights and fundamental freedoms,
        Whereas a common understanding of these rights and freedoms is of the greatest importance for the full realization of this pledge,

        Now, therefore,

        The General Assembly,

        Proclaims this Universal Declaration of Human Rights as a common standard of achievement for all peoples and all nations, to the end that every individual and every organ of society, keeping this Declaration constantly in mind, shall strive by teaching and education to promote respect for these rights and freedoms and by progressive measures, national and international, to secure their universal and effective recognition and observance, both among the peoples of Member States themselves and among the peoples of territories under their jurisdiction.

        Article 1
        All human beings are born free and equal in dignity and rights. They are endowed with reason and conscience and should act towards one another in a spirit of brotherhood.

        Article 2
        Everyone is entitled to all the rights and freedoms set forth in this Declaration, without distinction of any kind, such as race, colour, sex, language, religion, political or other opinion, national or social origin, property, birth or other status. Furthermore, no distinction shall be made on the basis of the political, jurisdictional or international status of the country or territory to which a person belongs, whether it be independent, trust, non-self-governing or under any other limitation of sovereignty.

        Article 3
        Everyone has the right to life, liberty and security of person.

        Article 4
        No one shall be held in slavery or servitude; slavery and the slave trade shall be prohibited in all their forms.

        Article 5
        No one shall be subjected to torture or to cruel, inhuman or degrading treatment or punishment.

        Article 6
        Everyone has the right to recognition everywhere as a person before the law.

        Article 7
        All are equal before the law and are entitled without any discrimination to equal protection of the law. All are entitled to equal protection against any discrimination in violation of this Declaration and against any incitement to such discrimination.

        Article 8
        Everyone has the right to an effective remedy by the competent national tribunals for acts violating the fundamental rights granted him by the constitution or by law.

        Article 9
        No one shall be subjected to arbitrary arrest, detention or exile.

        Article 10
        Everyone is entitled in full equality to a fair and public hearing by an independent and impartial tribunal, in the determination of his rights and obligations and of any criminal charge against him.



        Here are the details of the AI system:

        Domain: "{}",
        Purpose: "{}",
        Capability: "{}",
        Sensing instrument and location: "{}",
        AI User: "{}",
        AI Subject: "{}"

         Please return the assessment in the following format:
         {{
            "Description": "The AI system is intended to be used ...",
            "Assessment of impact on human rights":[
              {{
                "Article 1": "[Supports/Violates/Does not impact]",
                "Reasoning for Article 1": "[Explanation]",
                "Relevant text from Article 1": "[Text]"
              }},
              {{
                "Article 2": "[Supports/Violates/Does not impact]",
                "Reasoning for Article 2": "[Explanation]",
                "Relevant text from Article 2": "[Text]"
              }},
              ...
              {{
                "Article 9": "[Supports/Violates/Does not impact]",
                "Reasoning for Article 9": "[Explanation]",
                "Relevant text from Article 9": "[Text]"
              }},
              {{
                "Article 10": "[Supports/Violates/Does not impact]",
                "Reasoning for Article 10": "[Explanation]",
                "Relevant text from Article 10": "[Text]"
              }}
            ]
         }}


        Ensure to output a *correctly formatted* JSON!!!
        """
    }
]



def format_prompt(MESSAGES, domain,purpose,aiCapability,sensingInstrument,aiUser,aiSubject):
    S = "test {}"
    messages = deepcopy(MESSAGES)
    messages[1]['content'] = messages[1]['content'].format(domain,purpose,aiCapability,sensingInstrument,aiUser,aiSubject)
    return messages


In [ ]:
FULL_RES = []
cost = 0

start_time = time.time()
i = 0
for useElements in prompt_result:
  useI = str(useElements['Use'])
  if int(useI) < 80:
    continue
  print (f" Parsing use {useI}")

  # Variables for message placeholders
  domain = useElements['Domain']
  purpose = useElements['Purpose']
  aiCapability = useElements['Capability']
  sensingInstrument = useElements['Sensing instrument and location']
  aiUser = useElements['AI User']
  aiSubject = useElements['AI Subject']

  # Extracting "Use i" details
  use_i_details = [domain, purpose, aiCapability, sensingInstrument, aiUser, aiSubject]

  # adapt the prompt for useI
  messages = format_prompt(MESSAGES, domain,purpose,aiCapability,sensingInstrument,aiUser,aiSubject)

  # run the prompt
  response = get_completion_from_messages(messages, temperature=0)
  print(response)

  # response, token_count = get_completion_and_token_count(messages, temperature=0)
  # res = token_count
  # cost_chunk = (res['prompt_tokens'] * 0.03  + res['completion_tokens'] * 0.06)/1000.0
  # cost += cost_chunk

  response = ast.literal_eval(response)

  # print(response)

  # combine the useI and the risk report
  combined_response = {}
  combined_response["id"]= useI
  combined_response["Details"] = use_i_details
  for k, v in response.items():
    combined_response[k] = v
  print (combined_response)

  # save result
  # with open(f"{useI}_risk_report_full.json", "w") as json_file:
  #     json.dump(combined_response, json_file, indent=4)  # 4 spaces of indentation
  # # Download the file to your local machine
  # files.download(f"{useI}_risk_report_full.json")

  FULL_RES.append(combined_response)

  time.sleep(44)

  i+=1
  # print(i)
  # UNCOMMENT FOR RUNNING FOR 3 uses only
  # if i==25:
  #   break

FULL_RES_part1 = FULL_RES.copy()

###############################
# save result
# with open(f"FULL_SDG_REPORT_EO.json", "w") as json_file:
#     json.dump(FULL_RES, json_file, indent=4)  # 4 spaces of indentation
# # Download the file to your local machine
# files.download(f"FULL_SDG_REPORT_EO.json")
# end_time = time.time()

# print(f"Execution time: {end_time - start_time:.5f} seconds")
# print (f"TOTAL COST {cost}")

 Parsing use 80
{
    "Description": "The AI system is designed to assess potential sites for renewable energy using satellite-based multispectral sensors, providing site suitability analysis for energy companies and impacting local communities.",
    "Assessment of impact on human rights":[
      {
        "Article 1": "Does not impact",
        "Reasoning for Article 1": "The AI system does not directly impact the inherent freedom and equality in dignity and rights of all human beings. It neither promotes nor hinders the spirit of brotherhood among individuals.",
        "Relevant text from Article 1": "All human beings are born free and equal in dignity and rights. They are endowed with reason and conscience and should act towards one another in a spirit of brotherhood."
      },
      {
        "Article 2": "Does not impact",
        "Reasoning for Article 2": "The AI system does not discriminate based on race, colour, sex, language, religion, political or other opinion, national o

In [ ]:
###############################
# save result
with open(f"HR_1_10_EO_uses_79_138.json", "w") as json_file:
    json.dump(FULL_RES, json_file, indent=4)  # 4 spaces of indentation
# Download the file to your local machine
files.download(f"HR_1_10_EO_uses_79_138.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# PART 2 PROMPT : HR 11-20

In [ ]:
# Assuming you have the variables domain, purpose, aiCapability, aiUser, and aiSubject defined with appropriate values

MESSAGES = [
    {
        'role': 'system',
        'content': """You are a renowned specialist in the field of Earth Observation technology with a dedicated focus on understanding, promoting, and supporting the Universal Human Rights. With your vast experience, decisiveness, and conscientious approach, you have a deep understanding of how Earth Observation technologies can be leveraged to impact the Human Rights.
        You possess comprehensive knowledge of the List of human rights definitions, which encapsulates all 30 articles from the UN Universal Decalartion on Human Rights. This global framework was meticulously crafted by the United Nations.
        """
    },
    {
        'role': 'user',
        'content': """

        Assess the AI system's alignment with the Universal Declaration of Human Rights. Specifically, for each of the human rights listed and explained below, assess if the AI system supports it, violates it, or does not impact it.

        Follow the streamlined steps below:
        1. Describe the AI System: Craft a concise description of the AI system, ensuring it parallels the phrasing used in the Sustainable Development Goal targets and indicators. Your description should begin with "The AI system is designed to..." and should be encapsulated within two sentences.
        2. Evaluate Each human right (described in the article). Classify the impact on human rights as "Supports", "Violates", or "Does not impact". To do so, assess if the AI system:
            i) Supports it, i.e., presents an opportunity for or supports the respective human right,
            ii) Violates it, i.e., presents a risk for the respective human right,
            iii) Does not impact it, i.e., there is no clear link that the system supports or violated the respective human right.

        3. In each case, rigorously validate your reasoning. While evaluating, explicitly cite the article text that resonate most with the AI system's purpose and capabilities. Reference the exact text from the respective human right article and provide a thorough rationale. The true intent and capabilities of the AI system are crucial for this evaluation. Ensure there is a direct correlation between the system's functionalities and the referenced article.

        This assessment requires a deep understanding of both the AI system in question and the articles of the Universal Declaration of Human Rights. Each evaluation must be backed by solid reasoning and a clear understanding of the interplay between technology and human rights.
        Remember, precision is paramount. Be succinct.


        *** Universal Declaration of Human Rights ***

        Preamble
        Whereas recognition of the inherent dignity and of the equal and inalienable rights of all members of the human family is the foundation of freedom, justice and peace in the world,
        Whereas disregard and contempt for human rights have resulted in barbarous acts which have outraged the conscience of mankind, and the advent of a world in which human beings shall enjoy freedom of speech and belief and freedom from fear and want has been proclaimed as the highest aspiration of the common people,
        Whereas it is essential, if man is not to be compelled to have recourse, as a last resort, to rebellion against tyranny and oppression, that human rights should be protected by the rule of law,
        Whereas it is essential to promote the development of friendly relations between nations,
        Whereas the peoples of the United Nations have in the Charter reaffirmed their faith in fundamental human rights, in the dignity and worth of the human person and in the equal rights of men and women and have determined to promote social progress and better standards of life in larger freedom,
        Whereas Member States have pledged themselves to achieve, in co-operation with the United Nations, the promotion of universal respect for and observance of human rights and fundamental freedoms,
        Whereas a common understanding of these rights and freedoms is of the greatest importance for the full realization of this pledge,

        Now, therefore,

        The General Assembly,

        Proclaims this Universal Declaration of Human Rights as a common standard of achievement for all peoples and all nations, to the end that every individual and every organ of society, keeping this Declaration constantly in mind, shall strive by teaching and education to promote respect for these rights and freedoms and by progressive measures, national and international, to secure their universal and effective recognition and observance, both among the peoples of Member States themselves and among the peoples of territories under their jurisdiction.

        Article 11
        Everyone charged with a penal offence has the right to be presumed innocent until proved guilty according to law in a public trial at which he has had all the guarantees necessary for his defence.
        No one shall be held guilty of any penal offence on account of any act or omission which did not constitute a penal offence, under national or international law, at the time when it was committed. Nor shall a heavier penalty be imposed than the one that was applicable at the time the penal offence was committed.
        Article 12
        No one shall be subjected to arbitrary interference with his privacy, family, home or correspondence, nor to attacks upon his honour and reputation. Everyone has the right to the protection of the law against such interference or attacks.

        Article 13
        Everyone has the right to freedom of movement and residence within the borders of each state.
        Everyone has the right to leave any country, including his own, and to return to his country.

        Article 14
        Everyone has the right to seek and to enjoy in other countries asylum from persecution.
        This right may not be invoked in the case of prosecutions genuinely arising from non-political crimes or from acts contrary to the purposes and principles of the United Nations.

        Article 15
        Everyone has the right to a nationality.
        No one shall be arbitrarily deprived of his nationality nor denied the right to change his nationality.

        Article 16
        Men and women of full age, without any limitation due to race, nationality or religion, have the right to marry and to found a family. They are entitled to equal rights as to marriage, during marriage and at its dissolution.
        Marriage shall be entered into only with the free and full consent of the intending spouses.
        The family is the natural and fundamental group unit of society and is entitled to protection by society and the State.

        Article 17
        Everyone has the right to own property alone as well as in association with others.
        No one shall be arbitrarily deprived of his property.

        Article 18
        Everyone has the right to freedom of thought, conscience and religion; this right includes freedom to change his religion or belief, and freedom, either alone or in community with others and in public or private, to manifest his religion or belief in teaching, practice, worship and observance.

        Article 19
        Everyone has the right to freedom of opinion and expression; this right includes freedom to hold opinions without interference and to seek, receive and impart information and ideas through any media and regardless of frontiers.

        Article 20
        Everyone has the right to freedom of peaceful assembly and association.
        No one may be compelled to belong to an association.



        Here are the details of the AI system:

        Domain: "{}",
        Purpose: "{}",
        Capability: "{}",
        Sensing instrument and location: "{}",
        AI User: "{}",
        AI Subject: "{}"

         Please return the assessment in the following format:
         {{
            "Description": "The AI system is intended to be used ...",
            "Assessment of impact on human rights":[
              {{
                "Article 11": "[Supports/Violates/Does not impact]",
                "Reasoning for Article 11": "[Explanation]",
                "Relevant text from Article 11": "[Text]"
              }},
              {{
                "Article 12": "[Supports/Violates/Does not impact]",
                "Reasoning for Article 12": "[Explanation]",
                "Relevant text from Article 12": "[Text]"
              }},
              ...
              {{
                "Article 19": "[Supports/Violates/Does not impact]",
                "Reasoning for Article 19": "[Explanation]",
                "Relevant text from Article 19": "[Text]"
              }},
              {{
                "Article 20": "[Supports/Violates/Does not impact]",
                "Reasoning for Article 20": "[Explanation]",
                "Relevant text from Article 20": "[Text]"
              }}
            ]
         }}


        Ensure to output a *correctly formatted* JSON!!!
        """
    }
]



def format_prompt(MESSAGES, domain,purpose,aiCapability,sensingInstrument,aiUser,aiSubject):
    S = "test {}"
    messages = deepcopy(MESSAGES)
    messages[1]['content'] = messages[1]['content'].format(domain,purpose,aiCapability,sensingInstrument,aiUser,aiSubject)
    return messages


In [ ]:
FULL_RES = []
cost = 0

start_time = time.time()
i = 0
for useElements in prompt_result:
  useI = str(useElements['Use'])
  if int(useI) < 114:
    continue
  # print (f" Parsing use {useI}")

  # Variables for message placeholders
  domain = useElements['Domain']
  purpose = useElements['Purpose']
  aiCapability = useElements['Capability']
  sensingInstrument = useElements['Sensing instrument and location']
  aiUser = useElements['AI User']
  aiSubject = useElements['AI Subject']

  # Extracting "Use i" details
  use_i_details = [domain, purpose, aiCapability, sensingInstrument, aiUser, aiSubject]

  # adapt the prompt for useI
  messages = format_prompt(MESSAGES, domain,purpose,aiCapability,sensingInstrument,aiUser,aiSubject)

  # run the prompt
  response = get_completion_from_messages(messages, temperature=0)
  # print(response)

  # response, token_count = get_completion_and_token_count(messages, temperature=0)
  # res = token_count
  # cost_chunk = (res['prompt_tokens'] * 0.03  + res['completion_tokens'] * 0.06)/1000.0
  # cost += cost_chunk

  response = ast.literal_eval(response)

  # print(response)

  # combine the useI and the risk report
  combined_response = {}
  combined_response["id"]= useI
  combined_response["Details"] = use_i_details
  for k, v in response.items():
    combined_response[k] = v
  pprint.pprint (combined_response)

  # save result
  # with open(f"{useI}_risk_report_full.json", "w") as json_file:
  #     json.dump(combined_response, json_file, indent=4)  # 4 spaces of indentation
  # # Download the file to your local machine
  # files.download(f"{useI}_risk_report_full.json")

  FULL_RES.append(combined_response)

  time.sleep(33)

  i+=1
  # print(i)
  # UNCOMMENT FOR RUNNING FOR 3 uses only
  # if i==25:
  #   break

FULL_RES_part2 = FULL_RES.copy()

###############################
# save result
# with open(f"FULL_SDG_REPORT_EO.json", "w") as json_file:
#     json.dump(FULL_RES, json_file, indent=4)  # 4 spaces of indentation
# # Download the file to your local machine
# files.download(f"FULL_SDG_REPORT_EO.json")
# end_time = time.time()

# print(f"Execution time: {end_time - start_time:.5f} seconds")
# print (f"TOTAL COST {cost}")

{'Description': 'The AI system is designed to assess water quality through the '
                'detection of water pollution. It leverages multispectral '
                'satellites to gather data, which is then used by '
                'environmental agencies and water management authorities to '
                'make informed decisions about water safety and sustainability '
                'for local communities and aquatic life.',
 'Details': ['Environment and Sustainability',
             'Assessing water quality',
             'Water pollution detection',
             'Multispectral satellites',
             'Environmental agencies, water management authorities',
             'Local communities, aquatic life'],
 'Indicators Supported for SDG 10': 'Not Relevant',
 'Indicators Supported for SDG 11': ['11.6.2'],
 'Indicators Supported for SDG 6': ['6.3.2', '6.6.1'],
 'Indicators Supported for SDG 7': 'Not Relevant',
 'Indicators Supported for SDG 8': 'Not Relevant',
 'Indicators

In [ ]:
###############################
# save result
with open(f"FULL_SDG_REPORT_EO_part2.json", "w") as json_file:
    json.dump(FULL_RES, json_file, indent=4)  # 4 spaces of indentation
# Download the file to your local machine
files.download(f"FULL_SDG_REPORT_EO_part2.json")

NameError: ignored

# PART 3 PROMPT : HR 21-30

In [ ]:
# Assuming you have the variables domain, purpose, aiCapability, aiUser, and aiSubject defined with appropriate values

MESSAGES = [
    {
        'role': 'system',
        'content': """You are a renowned specialist in the field of Earth Observation technology with a dedicated focus on understanding, promoting, and supporting the Universal Human Rights. With your vast experience, decisiveness, and conscientious approach, you have a deep understanding of how Earth Observation technologies can be leveraged to impact the Human Rights.
        You possess comprehensive knowledge of the List of human rights definitions, which encapsulates all 30 articles from the UN Universal Decalartion on Human Rights. This global framework was meticulously crafted by the United Nations.
        """
    },
    {
        'role': 'user',
        'content': """

        Assess the AI system's alignment with the Universal Declaration of Human Rights. Specifically, for each of the human rights listed and explained below, assess if the AI system supports it, violates it, or does not impact it.

        Follow the streamlined steps below:
        1. Describe the AI System: Craft a concise description of the AI system, ensuring it parallels the phrasing used in the Sustainable Development Goal targets and indicators. Your description should begin with "The AI system is designed to..." and should be encapsulated within two sentences.
        2. Evaluate Each human right (described in the article). Classify the impact on human rights as "Supports", "Violates", or "Does not impact". To do so, assess if the AI system:
            i) Supports it, i.e., presents an opportunity for or supports the respective human right,
            ii) Violates it, i.e., presents a risk for the respective human right,
            iii) Does not impact it, i.e., there is no clear link that the system supports or violated the respective human right.

        3. In each case, rigorously validate your reasoning. While evaluating, explicitly cite the article text that resonate most with the AI system's purpose and capabilities. Reference the exact text from the respective human right article and provide a thorough rationale. The true intent and capabilities of the AI system are crucial for this evaluation. Ensure there is a direct correlation between the system's functionalities and the referenced article.

        This assessment requires a deep understanding of both the AI system in question and the articles of the Universal Declaration of Human Rights. Each evaluation must be backed by solid reasoning and a clear understanding of the interplay between technology and human rights.
        Remember, precision is paramount. Be succinct.


        *** Universal Declaration of Human Rights ***

        Preamble
        Whereas recognition of the inherent dignity and of the equal and inalienable rights of all members of the human family is the foundation of freedom, justice and peace in the world,
        Whereas disregard and contempt for human rights have resulted in barbarous acts which have outraged the conscience of mankind, and the advent of a world in which human beings shall enjoy freedom of speech and belief and freedom from fear and want has been proclaimed as the highest aspiration of the common people,
        Whereas it is essential, if man is not to be compelled to have recourse, as a last resort, to rebellion against tyranny and oppression, that human rights should be protected by the rule of law,
        Whereas it is essential to promote the development of friendly relations between nations,
        Whereas the peoples of the United Nations have in the Charter reaffirmed their faith in fundamental human rights, in the dignity and worth of the human person and in the equal rights of men and women and have determined to promote social progress and better standards of life in larger freedom,
        Whereas Member States have pledged themselves to achieve, in co-operation with the United Nations, the promotion of universal respect for and observance of human rights and fundamental freedoms,
        Whereas a common understanding of these rights and freedoms is of the greatest importance for the full realization of this pledge,

        Now, therefore,

        The General Assembly,

        Proclaims this Universal Declaration of Human Rights as a common standard of achievement for all peoples and all nations, to the end that every individual and every organ of society, keeping this Declaration constantly in mind, shall strive by teaching and education to promote respect for these rights and freedoms and by progressive measures, national and international, to secure their universal and effective recognition and observance, both among the peoples of Member States themselves and among the peoples of territories under their jurisdiction.

        Article 21
        Everyone has the right to take part in the government of his country, directly or through freely chosen representatives.
        Everyone has the right of equal access to public service in his country.
        The will of the people shall be the basis of the authority of government; this will shall be expressed in periodic and genuine elections which shall be by universal and equal suffrage and shall be held by secret vote or by equivalent free voting procedures.

        Article 22
        Everyone, as a member of society, has the right to social security and is entitled to realization, through national effort and international co-operation and in accordance with the organization and resources of each State, of the economic, social and cultural rights indispensable for his dignity and the free development of his personality.

        Article 23
        Everyone has the right to work, to free choice of employment, to just and favourable conditions of work and to protection against unemployment.
        Everyone, without any discrimination, has the right to equal pay for equal work.
        Everyone who works has the right to just and favourable remuneration ensuring for himself and his family an existence worthy of human dignity, and supplemented, if necessary, by other means of social protection.
        Everyone has the right to form and to join trade unions for the protection of his interests.

        Article 24
        Everyone has the right to rest and leisure, including reasonable limitation of working hours and periodic holidays with pay.

        Article 25
        Everyone has the right to a standard of living adequate for the health and well-being of himself and of his family, including food, clothing, housing and medical care and necessary social services, and the right to security in the event of unemployment, sickness, disability, widowhood, old age or other lack of livelihood in circumstances beyond his control.
        Motherhood and childhood are entitled to special care and assistance. All children, whether born in or out of wedlock, shall enjoy the same social protection.

        Article 26
        Everyone has the right to education. Education shall be free, at least in the elementary and fundamental stages. Elementary education shall be compulsory. Technical and professional education shall be made generally available and higher education shall be equally accessible to all on the basis of merit.
        Education shall be directed to the full development of the human personality and to the strengthening of respect for human rights and fundamental freedoms. It shall promote understanding, tolerance and friendship among all nations, racial or religious groups, and shall further the activities of the United Nations for the maintenance of peace.
        Parents have a prior right to choose the kind of education that shall be given to their children.

        Article 27
        Everyone has the right freely to participate in the cultural life of the community, to enjoy the arts and to share in scientific advancement and its benefits.
        Everyone has the right to the protection of the moral and material interests resulting from any scientific, literary or artistic production of which he is the author.

        Article 28
        Everyone is entitled to a social and international order in which the rights and freedoms set forth in this Declaration can be fully realized.

        Article 29
        Everyone has duties to the community in which alone the free and full development of his personality is possible.
        In the exercise of his rights and freedoms, everyone shall be subject only to such limitations as are determined by law solely for the purpose of securing due recognition and respect for the rights and freedoms of others and of meeting the just requirements of morality, public order and the general welfare in a democratic society.
        These rights and freedoms may in no case be exercised contrary to the purposes and principles of the United Nations.

        Article 30
        Nothing in this Declaration may be interpreted as implying for any State, group or person any right to engage in any activity or to perform any act aimed at the destruction of any of the rights and freedoms set forth herein.





        Here are the details of the AI system:

        Domain: "{}",
        Purpose: "{}",
        Capability: "{}",
        Sensing instrument and location: "{}",
        AI User: "{}",
        AI Subject: "{}"

         Please return the assessment in the following format:
         {{
            "Description": "The AI system is intended to be used ...",
            "Assessment of impact on human rights":[
              {{
                "Article 21": "[Supports/Violates/Does not impact]",
                "Reasoning for Article 21": "[Explanation]",
                "Relevant text from Article 21": "[Text]"
              }},
              {{
                "Article 22": "[Supports/Violates/Does not impact]",
                "Reasoning for Article 22": "[Explanation]",
                "Relevant text from Article 22": "[Text]"
              }},
              ...
              {{
                "Article 29": "[Supports/Violates/Does not impact]",
                "Reasoning for Article 29": "[Explanation]",
                "Relevant text from Article 29": "[Text]"
              }},
              {{
                "Article 30": "[Supports/Violates/Does not impact]",
                "Reasoning for Article 30": "[Explanation]",
                "Relevant text from Article 30": "[Text]"
              }}
            ]
         }}


        Ensure to output a *correctly formatted* JSON!!!
        """
    }
]



def format_prompt(MESSAGES, domain,purpose,aiCapability,sensingInstrument,aiUser,aiSubject):
    S = "test {}"
    messages = deepcopy(MESSAGES)
    messages[1]['content'] = messages[1]['content'].format(domain,purpose,aiCapability,sensingInstrument,aiUser,aiSubject)
    return messages


In [ ]:
FULL_RES = []
cost = 0

start_time = time.time()
i = 0
for useElements in prompt_result:
  useI = str(useElements['Use'])
  if int(useI) not in [92]:
    continue
  # print (f" Parsing use {useI}")

  # Variables for message placeholders
  domain = useElements['Domain']
  purpose = useElements['Purpose']
  aiCapability = useElements['Capability']
  sensingInstrument = useElements['Sensing instrument and location']
  aiUser = useElements['AI User']
  aiSubject = useElements['AI Subject']

  # Extracting "Use i" details
  use_i_details = [domain, purpose, aiCapability, sensingInstrument, aiUser, aiSubject]

  # adapt the prompt for useI
  messages = format_prompt(MESSAGES, domain,purpose,aiCapability,sensingInstrument,aiUser,aiSubject)

  # run the prompt
  response = get_completion_from_messages(messages, temperature=0)
  # print(response)

  # response, token_count = get_completion_and_token_count(messages, temperature=0)
  # res = token_count
  # cost_chunk = (res['prompt_tokens'] * 0.03  + res['completion_tokens'] * 0.06)/1000.0
  # cost += cost_chunk

  try:
    response = ast.literal_eval(response)
  except:
    time.sleep(33)
    continue

  # print(response)

  # combine the useI and the risk report
  combined_response = {}
  combined_response["id"]= useI
  combined_response["Details"] = use_i_details
  for k, v in response.items():
    combined_response[k] = v
  pprint.pprint (combined_response)

  # save result
  # with open(f"{useI}_risk_report_full.json", "w") as json_file:
  #     json.dump(combined_response, json_file, indent=4)  # 4 spaces of indentation
  # # Download the file to your local machine
  # files.download(f"{useI}_risk_report_full.json")

  FULL_RES.append(combined_response)

  time.sleep(33)

  i+=1
  # print(i)
  # UNCOMMENT FOR RUNNING FOR 3 uses only
  # if i==25:
  #   break

FULL_RES_part3 = FULL_RES.copy()


###############################
# save result
# with open(f"FULL_SDG_REPORT_EO.json", "w") as json_file:
#     json.dump(FULL_RES, json_file, indent=4)  # 4 spaces of indentation
# # Download the file to your local machine
# files.download(f"FULL_SDG_REPORT_EO.json")
# end_time = time.time()

# print(f"Execution time: {end_time - start_time:.5f} seconds")
# print (f"TOTAL COST {cost}")

{'Description': 'The AI system is designed to assess global environmental '
                'changes through the use of satellite-based multispectral '
                'sensors. It is intended to be used by international '
                'organizations to detect global changes, thereby informing '
                'diplomacy and foreign policy decisions that affect the global '
                'community.',
 'Details': ['Diplomacy and Foreign Policy',
             'Assessing global environmental changes',
             'Global change detection',
             'Satellite-based multispectral sensors',
             'International organizations',
             'Global community'],
 'Indicators Supported for SDG 12': '12.2.1, 12.2.2, 12.4.1, 12.4.2, 12.8.1',
 'Indicators Supported for SDG 13': '13.1.1, 13.1.2, 13.2.1, 13.2.2, 13.3.1',
 'Indicators Supported for SDG 14': '14.1.1, 14.2.1, 14.3.1',
 'Indicators Supported for SDG 15': '15.1.1, 15.1.2, 15.2.1, 15.3.1',
 'Indicators Supported for SD

In [ ]:
###############################
# save result
with open(f"FULL_SDG_REPORT_EO_part3.json", "w") as json_file:
    json.dump(FULL_RES_part3, json_file, indent=4)  # 4 spaces of indentation
# Download the file to your local machine
files.download(f"FULL_SDG_REPORT_EO_part3.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
###############################
# save result
with open(f"FULL_SDG_REPORT_EO_part3.json", "w") as json_file:
    json.dump(FULL_RES, json_file, indent=4)  # 4 spaces of indentation
# Download the file to your local machine
files.download(f"FULL_SDG_REPORT_EO_part3.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# THE END

# New section

# New section